## Import Statements
### The libraries used are `pandas`, `pathlib`, `imblearn`, and `sklearn`. 

In [8]:
import pandas as pd
from pathlib import Path
from imblearn.metrics import classification_report_imbalanced
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier

## Import the data and create a dataframe
1. Use `pandas` and `pathlib` to read the `BankChurners2.csv` CSV from the `Resources` folder.
2. Sample the dataset to review the data.

In [9]:
starter_df = pd.read_csv(Path("c://users/ajcth/documents/github/bank_churn_project/Resources/BankChurnersPrimary.csv"))
starter_df.sample(5)

,CLIENTNUM,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,...,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2
2552,714120258,Existing Customer,37,M,2,Uneducated,Divorced,$80K - $120K,Blue,36,...,2281.0,1677,604.0,1.027,2735,58,1.071,0.735,0.000185,0.99982
4971,789670533,Existing Customer,41,F,2,High School,Married,$40K - $60K,Blue,26,...,1438.3,0,1438.3,0.660,4300,67,0.675,0.000,0.000054,0.99995
8533,825838908,Existing Customer,39,F,1,Uneducated,Single,Less than $40K,Blue,35,...,1961.0,0,1961.0,0.631,4846,92,0.840,0.000,0.000117,0.99988
6806,826808208,Existing Customer,60,F,0,High School,Single,$40K - $60K,Blue,56,...,2213.0,829,1384.0,0.639,4210,84,0.714,0.375,0.000168,0.99983
4832,718982058,Existing Customer,37,M,3,Graduate,Divorced,$40K - $60K,Blue,36,...,5987.0,1405,4582.0,0.952,4011,72,0.674,0.235,0.000044,0.99996


## Change some of the longer column names to make the original dataframe easily readable

In [10]:
replacement_column_names = {'Attrition_Flag': 'Status',
        'Customer_Age': 'Age',
        'Education_Level': 'Education',
        'Dependent_count': 'Dependents',
        'Income_Category': 'Income',
        'Card_Category': 'Card_Type',
        'Months_on_book': 'Tenure(month)'
        }
starter_df.rename(columns=replacement_column_names, inplace=True)

## Drop any Unknown or NaN values

In [11]:
starter_df = starter_df[starter_df != 'Unknown'].dropna()

## Remove any columns that won't be used.
### Naive Bayes columns can't be used (it's calculated from features and target).
### CLIENTNUM is irrelevant.

In [12]:
starter_df.drop(columns=[
'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2', 
'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1',
'CLIENTNUM'
], 
inplace= True)

## Evaluate the Education column for use as a feature
### The Education values are ranked from uneducated to doctorate, and is a gradient rather than ambiguous classifications. The ranks will be converted to numericals and specified as datatype `int`.
1. Uneducated = 0
2. Highschool = 1
3. College = 2
4. Graduate = 3
5. Post-Grad = 4
6. Doctorate = 5

In [13]:
starter_df = starter_df.replace({'Education' : {
'Uneducated': 0, 
'High School': 1, 
'College' : 2, 
'Graduate' :  3, 
'Post-Graduate' : 4, 
'Doctorate' : 5
}})
starter_df.Education = starter_df.Education.astype(int)

## Evaluate the Card Type column for use as a feature
### The Card Type values are ranked from blue to platinum, and is a gradient rather than ambiguous classifications. The ranks will be converted to numericals and specified as datatype `int`.
1. Blue = 0
2. Silver = 1
3. Gold = 2
4. Platinum = 3


In [14]:
starter_df = starter_df.replace({'Card_Type' : {
'Blue': 0, 
'Silver': 1, 
'Gold' : 2, 
'Platinum' : 3
}})
starter_df.Card_Type = starter_df.Card_Type.astype(int)

## Evaluate the Gender column for use as a feature
### Gender is a binary classification, so we don't need to OneHotEncode.
### The values will be converted to numericals and specified as datatype `int`.
1. Male = 0
2. Female = 1 


In [15]:
starter_df = starter_df.replace({'Gender' : {
'M': 0, 
'F': 1
}})
starter_df.Gender = starter_df.Gender.astype(int)

## Evaluate the Status column for use as the target
### Status is a binary classification, so we don't need to OneHotEncode.
### The values will be converted to numericals and specified as datatype `int`.
1. Attrited Customer = 0
2. Existing Customer = 1 

In [16]:
starter_df = starter_df.replace({'Status' : {
'Attrited Customer': 0, 
'Existing Customer': 1
}})
starter_df.GenStatuStatussder = starter_df.Status.astype(int)

c:\python_files\Anaconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  """


## Perform feature creation on the Dependents and Marital_Status features to create the Marital_Dependent_Ratio feature
### The values will be converted to numericals and specified as datatype `int`.
### Divorced and Single will be combined, as they are both single income sources.
1. Divorced = 1
2. Single = 1 
3. Married

## The formula for Marital_Dependent_Ratio is as follows:
( Marital_Status / (Dependents + 1) / 2 )
## The original Marital_Status and Dependents columns will be dropped in place of the Marital_Dependent_Ratio column.

In [17]:
starter_df = starter_df.replace({'Marital_Status' : {
'Divorced': 1, 
'Single': 1, 
'Married' : 2
}})
starter_df.Marital_Status = starter_df.Marital_Status.astype(int)
starter_df['Marital_Dependent_Ratio'] = ((starter_df['Marital_Status'] / (starter_df['Dependents'] + 1)) / 2).round(2)
starter_df.drop(columns=['Marital_Status', 'Dependents'], inplace=True)


## Perform feature creation on the Total_Trans_Amt and Total_Trans_Ct features to create the Avg_Trans_Value feature. Its a simple ratio.
### The original Total_Trans_Amt and Total_Trans_Ct columns will be dropped in place of the Avg_Trans_Value column.

In [18]:
starter_df['CreditUsage'] = (starter_df['Avg_Open_To_Buy'] / starter_df['Credit_Limit']).round(2)
starter_df.drop(columns=['Avg_Open_To_Buy', 'Credit_Limit'], inplace=True)

## Perform feature creation on the Total_Trans_Amt and Total_Trans_Ct features to create the Avg_Trans_Value feature. Its a simple ratio.
### The original Total_Trans_Amt and Total_Trans_Ct columns will be dropped in place of the Avg_Trans_Value column.

In [19]:
starter_df['Avg_Trans_Value'] = (starter_df['Total_Trans_Amt'] / starter_df['Total_Trans_Ct']).round(2)
starter_df.drop(columns=['Total_Trans_Amt', 'Total_Trans_Ct'], inplace=True)

## Perform feature creation on the Tenure(month) and Age features to create the Tenure_By_Age feature. Its a simple ratio.

### The original Tenure(month) and Age columns will be dropped in place of the Tenure_By_Age column.

In [20]:
#Do feature creation to make a ratio between age and tenure
starter_df['Tenure_By_Age'] = (starter_df['Tenure(month)'] / starter_df['Age']).round(2)
starter_df.drop(columns=['Tenure(month)', 'Age'], inplace=True)

## Evaluate the Income column for use as the target
### The Income values are ranked in blocks, and is a gradient rather than ambiguous classifications. The ranks will be converted to numericals and specified as datatype `int`.

In [21]:
starter_df = starter_df.replace({'Income' : {
'Less than $40K': 0, 
'$40K - $60K': 1, 
'$80K - $120K' : 2, 
'$60K - $80K' : 3, 
'$120K +' : 4
}})
starter_df.Income = starter_df.Income.astype(int)

## Review the new features

In [35]:
starter_df.dtypes

Status                        int64
Gender                        int32
Education                     int32
Income                        int32
Card_Type                     int32
Total_Relationship_Count      int64
Months_Inactive_12_mon        int64
Contacts_Count_12_mon         int64
Total_Revolving_Bal           int64
Total_Amt_Chng_Q4_Q1        float64
Total_Ct_Chng_Q4_Q1         float64
Avg_Utilization_Ratio       float64
Marital_Dependent_Ratio     float64
CreditUsage                 float64
Avg_Trans_Value             float64
Tenure_By_Age               float64
dtype: object

In [22]:
starter_df.sample(5)

,Status,Gender,Education,Income,Card_Type,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Total_Revolving_Bal,Total_Amt_Chng_Q4_Q1,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Marital_Dependent_Ratio,CreditUsage,Avg_Trans_Value,Tenure_By_Age
6885,1,1,3,1,0,6,4,2,632,0.702,0.660,0.045,0.17,0.95,56.95,0.87
9233,1,0,0,3,0,2,1,1,1337,0.734,0.600,0.076,0.17,0.92,148.38,0.65
7495,1,0,3,3,0,4,2,3,896,0.680,0.516,0.397,0.20,0.60,49.13,0.64
8479,1,1,3,1,0,2,1,1,0,0.728,0.708,0.000,0.10,1.00,54.80,0.53
2488,1,0,0,1,0,3,1,1,607,0.883,0.477,0.057,0.25,0.94,62.68,0.80


## Define our features(X) and our target(y)

In [23]:
y = starter_df['Status']
X = starter_df.drop(columns=['Status'])

## Split features and target in to training and testing sets

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

## Using StandardScaler, scale all numerical/float values that don't represent ambiguous categories
### StandardScaler will be used within ColumnTransformer. This allows for the scaling of specified columns within our X_train and X_test set.
It's critical that the StandardScaler is only fit to our X_train. Both X_train and X_test will be transformed, but the StandardScaler will only be fit to X_train.This prevents the model from 'cheating'.

In [25]:
scaler = StandardScaler()

col_tran= ColumnTransformer([
('Total_Revolving_Bal_Scaled', scaler, ['Total_Revolving_Bal']),
('Education_Scaled', scaler, ['Education']),
('Income_Scaled', scaler, ['Income']),
('Card_Type_Scaled', scaler, ['Card_Type']),
('Months_Inactive_12_mon_scaled', scaler, ['Months_Inactive_12_mon']),
('Credit_Usage_Scaled', scaler, ['Credit_Usage']),
('Avg_Trans_Value_Scaled', scaler, ['Avg_Trans_Value']),
('Tenure_By_Age_Scaled', scaler, ['Tenure_By_Age']),
('Avg_Util_Ratio_Scaled', scaler, ['Avg_Utilization_Ratio']),
('Total_Relationship_Count_Scaled', scaler, ['Total_Relationship_Count']),
('Contacts_Count_12_mon_scaled', scaler, ['Contacts_Count_12_mon']),
('Total_Amt_Chng_Q4_Q1_scaled', scaler, ['Total_Amt_Chng_Q4_Q1']),
('Total_Ct_Chng_Q4_Q1_scaled', scaler, ['Total_Ct_Chng_Q4_Q1']),
('Marital_Dependent_Ratio_Scaled', scaler, ['Marital_Dependent_Ratio'])
])

## Use SMOTE to add synthetic data and balance our target/feature value count
### This will only be done on the training data.

In [26]:
oversample = SMOTE()
X_train, y_train = oversample.fit_resample(X_train, y_train)

## Create an instance of the GradientBoostingClassifier model
### Hyperparameter tuning was mostly unhelpful; only two hyperparameters have been changed from default values.
1. n_estimators = 500
2. random_state= 2

In [27]:
gbc_model = GradientBoostingClassifier(
n_estimators=500,                 #default = 100    range = 1-inf
random_state= 2,                  #default = None   range = 1-inf
subsample= 1,                     #default = 1      range = 0.-1
min_samples_split = 2,            #default = 2      range = 2-inf
max_depth=3,                      #default = 3      range = 1-inf
min_impurity_decrease=0,          #default = 0      range = 0 - inf
min_samples_leaf = 1,             #default = 1      range = 1 - inf
min_weight_fraction_leaf = 0,     #default = 0      range =0 - 0.5
max_leaf_nodes = None,            #default = None   range = 2-inf
learning_rate = 0.1               #default=0.1      range 0.0-inf
)

## Fit the model on the training data

In [28]:
gbc_model.fit(X_train, y_train)

GradientBoostingClassifier(min_impurity_decrease=0, min_weight_fraction_leaf=0,
                           n_estimators=500, random_state=2, subsample=1)

## Make predictions on the test data

In [29]:
gbc_test_predictions = gbc_model.predict(X_test)

## Evaluate the model's performance

In [30]:
accuracy = accuracy_score(y_test, gbc_test_predictions)
print("Accuracy: {:.2f}%".format(accuracy * 100))

Accuracy: 91.47%


## Generate a confusion matrix

In [31]:
gbc_test_matrix = confusion_matrix(y_test, gbc_test_predictions)
print(gbc_test_matrix)

[[ 198   79]
 [  72 1422]]


## Generate a classification report

In [32]:
gbc_testing_report = classification_report(y_test, gbc_test_predictions)
print(gbc_testing_report)

              precision    recall  f1-score   support

           0       0.73      0.71      0.72       277
           1       0.95      0.95      0.95      1494

    accuracy                           0.91      1771
   macro avg       0.84      0.83      0.84      1771
weighted avg       0.91      0.91      0.91      1771



## Generate an imbalanced classification report

In [33]:
imb__gbc_testing_report = classification_report_imbalanced(y_test, gbc_test_predictions)
print(imb__gbc_testing_report)

                   pre       rec       spe        f1       geo       iba       sup

          0       0.73      0.71      0.95      0.72      0.82      0.66       277
          1       0.95      0.95      0.71      0.95      0.82      0.70      1494

avg / total       0.91      0.91      0.75      0.91      0.82      0.69      1771

